# Indexing and selecting data

This section will be focused on slice subsets of series and dataframe objects. Python has built in tools to do this that while capable are not the best for the job. Pandas has optimized methods to access data reccommended for production code.

# Different choices for indexing

Pandas supports 3 types of multi-axis indexing:
    .loc(), mainly for label based operations but boolean arrays are also possible with a KeyError being raised in cases when the item isn't found. Possible inputs include:
            
            a single label(e.g. 'avocados' or 3 which is interpreted as an index label)
        
            a list or array of labels (e.g. ['avocado', 'banana']
        
            Slices of objects with labels (e.g. 'avocado':'banana'), unlike normal python slicing operations both the start and stop are included however.
        
            A boolean array
        
            A callable function with a single argument the yields a valid indexing output from the above list.

    .iloc() is primarily a integer position based (0 to len(-1)) but also accepts boolean arrays. When an indexer is out of bounds .iloc() will raise an IndexError barring a slice indexer which can use out-of-bounds indexers. Valid inputs include:
            
            An integer (e.g.9)
            
            A list or array of integers (e.g. [3, 6, 2])
            
            A slice object with ints (e.g. 0:3)
            
            A boolean array
            
            Or a callable function with a single argument the yields a valid indexing output from the above list.
            
    .loc(), .iloc(), and [] all accept callable functions as indexers

When working will multiple axes, the following notation appplies. Null slices (':') can be used fpr any accesor but can also be left out (e.g. df.loc['b'] == df.loc['b', :, :]).

For a series object the format is s.loc[indexer].
For a DataFrame object the format is df.loc[row_indexer, column_indexer]
For a Panel object the format is p.loc[item_indexer, major_indexer, minor_indexer]

# Basics

The primary function of slicing with [] notation is to select lower dimensiional slices.
    for a series, series[label] returns a scalar value
    
    for a dataframe, df[colname], returns a series matching the colname
    
    for a panel, panel[itemname], returns a dataframe matching the itemname

In [1]:
#importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#constructing a simple dataframe to work with
index = list('abcde')
df = pd.DataFrame(np.random.randn(5, 3), index = index, columns = ['happy', 'sad', 'angry'])
df

,happy,sad,angry
a,0.538547,-1.347968,1.850177
b,-0.404629,0.608098,0.432825
c,0.809862,0.907196,-0.473732
d,-0.429442,-0.487332,0.774350
e,0.622115,-0.375299,-0.954194


In [3]:
#constructing a panel
panel = pd.Panel({'alpha': df, 'beta' : df - df['angry'].mean()})
panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 5 (major_axis) x 3 (minor_axis)
Items axis: alpha to beta
Major_axis axis: a to e
Minor_axis axis: happy to angry

In [4]:
#selecting a scalar index from a series from a datframe from a panel
df1 = panel['beta']
s = df1['happy']
s[2:4]

c    0.483977
d   -0.755327
Name: happy, dtype: float64

In [5]:
#we can also select multiple columns by passing in a list using [[]].
s = df[['angry', 'sad']]
s[0:3]

,angry,sad
a,1.850177,-1.347968
b,0.432825,0.608098
c,-0.473732,0.907196


This same process can also be used to set multiple columns.

In [6]:
df[['sad', 'angry']] = df1[['sad', 'angry']]
df

,happy,sad,angry
a,0.538547,-1.673853,1.524292
b,-0.404629,0.282213,0.106940
c,0.809862,0.581310,-0.799617
d,-0.429442,-0.813217,0.448465
e,0.622115,-0.701184,-1.280079


This might be useful for applying in-place transformations to a subset of columns. However, it is important to note that pandas aligns all axes when setting Series and Dataframe objects using .loc and .iloc.

The following fails to modify the df because column alignment preceeds value assignment.

In [7]:
#incorrect method
df.loc[:, ['sad', 'angry']] = df[['angry', 'sad']]
df

,happy,sad,angry
a,0.538547,-1.673853,1.524292
b,-0.404629,0.282213,0.106940
c,0.809862,0.581310,-0.799617
d,-0.429442,-0.813217,0.448465
e,0.622115,-0.701184,-1.280079


The correct method uses the raw values as follows

In [8]:
df.loc[:, ['sad', 'angry']] = df[['angry', 'sad']].values
df

,happy,sad,angry
a,0.538547,1.524292,-1.673853
b,-0.404629,0.106940,0.282213
c,0.809862,-0.799617,0.581310
d,-0.429442,0.448465,-0.813217
e,0.622115,-1.280079,-0.701184


# Attribute Access

Directly accessible attributes include an index of a series, a dataframe column, and a panel item. In the IPython environment you can also use tab completion the access these values.

In [9]:
#accesing a series attribute
s = df['happy']
s.b

-0.40462943534016804

In [10]:
#on a dataframe
df.sad

a    1.524292
b    0.106940
c   -0.799617
d    0.448465
e   -1.280079
Name: sad, dtype: float64

In [11]:
#on a panel
panel.beta

,happy,sad,angry
a,0.212662,-1.673853,1.524292
b,-0.730515,0.282213,0.106940
c,0.483977,0.581310,-0.799617
d,-0.755327,-0.813217,0.448465
e,0.296230,-0.701184,-1.280079


we can also use this process to modify existing objects

In [12]:
#modifying a series value
s.b = 2
s.b

2.0

In [13]:
#modifying column values
df.happy = list(range(len(df.index)))
df

,happy,sad,angry
a,0,1.524292,-1.673853
b,1,0.106940,0.282213
c,2,-0.799617,0.581310
d,3,0.448465,-0.813217
e,4,-1.280079,-0.701184


In [14]:
#to create a new column the notation is as follows
df['glad'] = df.happy - df.sad
df

,happy,sad,angry,glad
a,0,1.524292,-1.673853,-1.524292
b,1,0.106940,0.282213,0.893060
c,2,-0.799617,0.581310,2.799617
d,3,0.448465,-0.813217,2.551535
e,4,-1.280079,-0.701184,5.280079


Some caveats:
    This access only works when the index element is a valid python identifier.
    The attribute is not available if it conflicts with an existing method name like min or max
    It will also be unavaiable if it conflicts with the following lists: *index, major_axis, minor_axis, items, labels.*

when these cases occur, standard indexing is still valid.

We can also assign a dict to a row of a dataframe:

In [15]:
df.iloc[3] = {'happy':12, 'sad':13, 'angry':14, 'glad':6}
df

,happy,sad,angry,glad
a,0,1.52429,-1.67385,-1.52429
b,1,0.10694,0.282213,0.89306
c,2,-0.799617,0.58131,2.79962
d,happy,sad,angry,glad
e,4,-1.28008,-0.701184,5.28008


# slicing ranges

This section will focus on the [] operator

With a Series the [] operator uses the same syntax as when working with an ndarray.

In [16]:
# slicing out a series from df
s = df['angry']
#slicing a range
s[:3]

a    -1.67385
b    0.282213
c     0.58131
Name: angry, dtype: object

In [17]:
s[::3]

a   -1.67385
d      angry
Name: angry, dtype: object

In [18]:
s[::-2]

e   -0.701184
c     0.58131
a    -1.67385
Name: angry, dtype: object

setting works the same was as well

In [19]:
s2 = s.copy()
s2[3] = 6
s2

a    -1.67385
b    0.282213
c     0.58131
d           6
e   -0.701184
Name: angry, dtype: object

Slicing for a dataframe using the [] operator slices rows.

In [20]:
#slicing rows in a dataframe
df[2:4]

,happy,sad,angry,glad
c,2,-0.799617,0.58131,2.79962
d,happy,sad,angry,glad


# Selection by label

This section concerns the .loc accessor and other purely label based methods.

a few notes:
    chained assignment should be avoided
    slicers must be compatible or convertible with the index type or they will raise a type error (e.g. tryin to slice a datetime index with integers will raise this error)
    
To reiterate a few points. Purely label based indexing in pandas is a strict inclusion protocol. Slices must include the start bound and stop bound when present in the index. Integers, in this case, refer to labels and not positions.

The.loc method is the primary attribute used to do this. valid inputs include the following:
    a single label
    
    a list or array of labels
    
    a slice object with labels 'start':'finish'
    
    a boolean array
    
    a callable function

In [21]:
# generating a new dataframe to work with
df = pd.DataFrame(np.random.randn(6, 6), index = list('abcdef'), columns = [1, 2, 3, 4, 5, 6])
df

,1,2,3,4,5,6
a,0.042035,-0.375996,0.309608,-0.432163,-0.201469,0.303789
b,0.986600,0.291543,-0.217095,-1.483815,1.237254,-0.088895
c,-0.136822,-0.286917,-1.351576,0.123052,0.187885,0.773000
d,0.608353,-0.448301,1.764878,0.466407,-0.902956,0.156088
e,0.884426,0.685086,1.449223,-0.386931,-0.507092,-0.374985
f,0.933914,-1.335786,0.286119,-1.714607,0.298713,-0.196829


In [22]:
#selecting rows based on a series label within a dataframe
df[1].loc['a':'c']

a    0.042035
b    0.986600
c   -0.136822
Name: 1, dtype: float64

In [23]:
#setting a series value based on label in a series within a dataframe
df[1].loc['a'] = np.nan
df

,1,2,3,4,5,6
a,NaN,-0.375996,0.309608,-0.432163,-0.201469,0.303789
b,0.986600,0.291543,-0.217095,-1.483815,1.237254,-0.088895
c,-0.136822,-0.286917,-1.351576,0.123052,0.187885,0.773000
d,0.608353,-0.448301,1.764878,0.466407,-0.902956,0.156088
e,0.884426,0.685086,1.449223,-0.386931,-0.507092,-0.374985
f,0.933914,-1.335786,0.286119,-1.714607,0.298713,-0.196829


In [24]:
#using selected rows and columns
df.loc[['b', 'd', 'f'], 1:3]

,1,2,3
b,0.986600,0.291543,-0.217095
d,0.608353,-0.448301,1.764878
f,0.933914,-1.335786,0.286119


In [25]:
#using label slices
df.loc['c':'f', 3:6]

,3,4,5,6
c,-1.351576,0.123052,0.187885,0.773000
d,1.764878,0.466407,-0.902956,0.156088
e,1.449223,-0.386931,-0.507092,-0.374985
f,0.286119,-1.714607,0.298713,-0.196829


In [26]:
#cross section with a label
df.loc['e']

1    0.884426
2    0.685086
3    1.449223
4   -0.386931
5   -0.507092
6   -0.374985
Name: e, dtype: float64

In [27]:
#using a boolean array
df.loc['a':'c', 1:4] <1

,1,2,3,4
a,False,True,True,True
b,True,True,True,True
c,True,True,True,True


In [28]:
#for grabbing a value explicitly, equivalent to "df.at['a', '1']
df.loc['b', 3]

-0.21709533642423476

# slicing with labels

Slicing using the .loc accessor returns the elements inbetween and including the start and stop labels when they are both present in the index.

In [29]:
s = df[1]
s

a         NaN
b    0.986600
c   -0.136822
d    0.608353
e    0.884426
f    0.933914
Name: 1, dtype: float64

In [30]:
#slicing a series
s.loc['b':'e']

b    0.986600
c   -0.136822
d    0.608353
e    0.884426
Name: 1, dtype: float64

In the case where one of the two elements is missing but the index is sorted. Slicing will still work by selecting the labels ranked between the two.

In [31]:
s.sort_index().loc['d':'g']

d    0.608353
e    0.884426
f    0.933914
Name: 1, dtype: float64

In this same case when the index is not sorted an error will be raised instead. aka, don't do the thing.

# Selecting by Position

Chained assingment should be avoided

Purely integer based indexing is available in pandas with a number of methods. All methods are 0 based, start bound is included stop bound is excluded. Using anything but an integer will raise and IndexError.

.iloc is the primary method with the following valid inputs
    an integer
    a list or array of integers
    a slice object with ints

In [32]:
# creating a new series to work with
s = pd.Series(np.random.randn(6), index = list(range(0, 18, 3)))
s

0     2.179733
3    -0.401220
6     0.575178
9     0.254726
12   -2.494186
15    0.314692
dtype: float64

In [33]:
#grabbing the third value (integer position 2 in this case)
s.iloc[2]

0.5751782990746216

In [34]:
#grabbing a central slice
s.iloc[2:4]

6    0.575178
9    0.254726
dtype: float64

In [35]:
#setting the value of integer position 2
s.iloc[2] = 3
s

0     2.179733
3    -0.401220
6     3.000000
9     0.254726
12   -2.494186
15    0.314692
dtype: float64

In [36]:
#generating a new dataframe to work with
df = pd.DataFrame(np.random.randn(10, 5), index = list(range(0, 40, 4)), columns = list(range(0, 10, 2)))
df

,0,2,4,6,8
0,-1.419713,1.237041,-0.186854,0.108618,0.456425
4,-2.366868,1.212321,0.088993,-2.107364,-1.282272
8,-1.669043,0.755079,-0.811231,2.968224,-2.325960
12,0.991108,-0.558594,0.113017,-0.059008,1.759883
16,1.245291,-1.124675,0.182869,1.083892,-0.850049
20,-0.577441,0.499518,-1.060338,1.031951,-1.309618
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779
28,-1.474076,-0.582585,-2.159539,0.167656,-1.276778
32,0.125106,0.587651,-2.095101,-0.427217,-0.435834
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711


In [37]:
#using integer slicing, specifying rows
df.iloc[:4]

,0,2,4,6,8
0,-1.419713,1.237041,-0.186854,0.108618,0.456425
4,-2.366868,1.212321,0.088993,-2.107364,-1.282272
8,-1.669043,0.755079,-0.811231,2.968224,-2.325960
12,0.991108,-0.558594,0.113017,-0.059008,1.759883


In [38]:
#using integer slicing specifying rows and columns
df.iloc[3:5, 3:5]

,6,8
12,-0.059008,1.759883
16,1.083892,-0.850049


In [39]:
#using a list of integers
df.iloc[[3, 5, 6], [3, 4]]

,6,8
12,-0.059008,1.759883
20,1.031951,-1.309618
24,-0.695691,0.707779


basic form for dataframes
df.iloc[rows, columns]

When a cross section is desired:

In [40]:
df.iloc[3]

0    0.991108
2   -0.558594
4    0.113017
6   -0.059008
8    1.759883
Name: 12, dtype: float64

In [41]:
#out of bounds should be handled as well
df.iloc[7:15, 3:10]

,6,8
28,0.167656,-1.276778
32,-0.427217,-0.435834
36,0.847664,1.248711


when slices go out of bounds they can result in an empty dataframe.

when a single indexer is out of bounds an IndexError will be raised. Similarly, a list of indexers where any single element is out of bounds will also raise an IndexError

# Selection by a callable

The primary indexers (.loc, .iloc, and []) can all accept a callable function as an indexer. However, the callable MUST be a function with one argument(the data object in this case), returning a valid output for indexing.

In [42]:
df.loc[lambda df: df[0] > -1]

,0,2,4,6,8
12,0.991108,-0.558594,0.113017,-0.059008,1.759883
16,1.245291,-1.124675,0.182869,1.083892,-0.850049
20,-0.577441,0.499518,-1.060338,1.031951,-1.309618
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779
32,0.125106,0.587651,-2.095101,-0.427217,-0.435834
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711


In [43]:
df.iloc[:, lambda df: [2, 3] ]

,4,6
0,-0.186854,0.108618
4,0.088993,-2.107364
8,-0.811231,2.968224
12,0.113017,-0.059008
16,0.182869,1.083892
20,-1.060338,1.031951
24,2.084870,-0.695691
28,-2.159539,0.167656
32,-2.095101,-0.427217
36,-0.076181,0.847664


In [44]:
df[lambda df: df.columns[:3]]

,0,2,4
0,-1.419713,1.237041,-0.186854
4,-2.366868,1.212321,0.088993
8,-1.669043,0.755079,-0.811231
12,0.991108,-0.558594,0.113017
16,1.245291,-1.124675,0.182869
20,-0.577441,0.499518,-1.060338
24,-0.078035,-0.418217,2.084870
28,-1.474076,-0.582585,-2.159539
32,0.125106,0.587651,-2.095101
36,-0.509459,-0.280707,-0.076181


In [45]:
#callable indexing can also be used in a series
s.loc[lambda s: s>1]

0    2.179733
6    3.000000
dtype: float64

you can avoid the use of a temporary variable by using chain data selection operations


The following are deprecated
.ix indexer in favor of .loc, and .iloc
using .loc or [] with a list containing one or more missing labels in favor of .reindex()

# reindexing

This is the idiomatic way to select potentially not-found elements.

In [46]:
s.reindex([0, 3, 8])

0    2.179733
3   -0.401220
8         NaN
dtype: float64

Another option for returing only valid keys and preserving the dtype is the following:

In [47]:
labels = [0, 3, 6]
s.loc[s.index.intersection(labels)]

0    2.179733
3   -0.401220
6    3.000000
dtype: float64

a duplicated index will raise an error for .reindex()

In [48]:
#generating a new series and failing to generate the the dulicated axis error
s = pd.Series(np.arange(5), index = ['a', 'b', 'c', 'd', 'e'])
labels = ['a', 'b']
s.reindex(labels)

a    0
b    1
dtype: int32

The duplication error can be circumvented by first intersecting the desired lables and then reindexing but this will still raise an error if the resulting index is duplicated

# selecting random samples

This is done using the sample() method on a Series, DataFrame or Panel. Its default behavior is to sample rows by default, returning a specific number of rows/columns or a fraction of rows

In [49]:
#generating a new series to work with
s = pd.Series(np.arange(15), index = list('abcdefghijklmno'))
print(s)

a     0
b     1
c     2
d     3
e     4
f     5
g     6
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
dtype: int32


In [50]:
#without passing an argument only one row is returned
s.sample()

i    8
dtype: int32

In [51]:
#specifying a number of rows
s.sample(n=5)

j     9
h     7
n    13
m    12
l    11
dtype: int32

In [52]:
#sampling a fraction of rows
s.sample(frac = 0.66)

h     7
o    14
c     2
j     9
b     1
f     5
i     8
a     0
n    13
l    11
dtype: int32

You can sample with replacement using the replace option otherwise sample() will only return each row at most once.

In [53]:
#sampling without replacement
s.sample(n = 5, replace=False)

i    8
f    5
e    4
c    2
g    6
dtype: int32

In [54]:
#with replacement
s.sample(n=6, replace = True)

m    12
j     9
g     6
n    13
d     3
c     2
dtype: int32

Using sample(), by default, each row has an equal probability of being selected. To change this we can pass the weights argument to the sample function. The weights can be in the form of a list, a np array, or series as long as they are the same length as the object being sampled. Missing values are assigned a weight of 0 and infinite values are not allowed. If the weights do not sum to one they will be normalized by dividing all the weights by the sum of the weights.

In [55]:
#creating a list of weights with sum 105
weights = pd.Series(np.arange(15))
#sampling with weights with re-normalizing
s.sample(n= 5, weights = weights.values)

o    14
l    11
m    12
e     4
j     9
dtype: int32

For DataFrames, a column within the df can be used as sampling weights(only when you are sampling rows, not when you are sampling columns) by passing the column name as a string.

In [56]:
#modifying our existing dataframe
df['weights'] = df[8]
del df[8]

In [57]:
#selecting the rows where weights are positive since it cannot accept negatives
df = df[df['weights'] >=0]

In [58]:
#sampling
df.sample(n = 4, weights = 'weights')

,0,2,4,6,weights
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711
12,0.991108,-0.558594,0.113017,-0.059008,1.759883
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779
0,-1.419713,1.237041,-0.186854,0.108618,0.456425


In [59]:
#we can alse sample columns
df.sample(n = 2, axis = 1)

,weights,2
0,0.456425,1.237041
12,1.759883,-0.558594
24,0.707779,-0.418217
36,1.248711,-0.280707


As a final not, we can also set a seed for sample()'s RNG using the random_state arg using either an int or a np RandomState object.

In [60]:
#the sample will always draw the same rows when given a seed(aka. int)
df.sample(n=3, random_state= 5)

,0,2,4,6,weights
0,-1.419713,1.237041,-0.186854,0.108618,0.456425
12,0.991108,-0.558594,0.113017,-0.059008,1.759883
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779


# Setting with Enlargement
enlargement can be performed using either the .loc or [] operations when setting a non-existent key for that axis.

This is basically an appending operation in the case of a series

In [61]:
# setting by enlargement a value for p
s['p'] = 15
s

a     0
b     1
c     2
d     3
e     4
f     5
g     6
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
p    15
dtype: int64

In the case of a DataFrame, either axis can be enlarged by using .loc

In [62]:
#creating a new column via enlargement using the .loc accessor
df.loc[:, 'alpha'] = 15
df
#I am not sure about the error here......

C:\Users\Talimenios\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Talimenios\Anaconda3\lib\site-packages\pandas\core\indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,0,2,4,6,weights,alpha
0,-1.419713,1.237041,-0.186854,0.108618,0.456425,15
12,0.991108,-0.558594,0.113017,-0.059008,1.759883,15
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779,15
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711,15


In [63]:
#the following is an append operation
df.loc[8, :] = 7
df

C:\Users\Talimenios\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,0,2,4,6,weights,alpha
0,-1.419713,1.237041,-0.186854,0.108618,0.456425,15.0
12,0.991108,-0.558594,0.113017,-0.059008,1.759883,15.0
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779,15.0
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711,15.0
8,7.000000,7.000000,7.000000,7.000000,7.000000,7.0


# Fast scalar value getting and setting

indexing with [] handles multiple use cases and as such it has some overhead to understand what you're asking for. To access only scalar values it is therefore advisable to utilize the specialized at(labels) and iat(integers) methods

In [64]:
#using .iat[]
s.iat[3]

3

In [65]:
#using .at on a dataframe, format [row, column]
df.at[8, 'alpha']

7.0

In [66]:
#using .iat[] to access the same value
df.iat[1, 5]

15.0

In [67]:
#it is also possible to set scalar values using these indexers
#changing the value of [8, 'alpha'] from 7 to 14
df.at[8, 'alpha'] = 14
df

,0,2,4,6,weights,alpha
0,-1.419713,1.237041,-0.186854,0.108618,0.456425,15.0
12,0.991108,-0.558594,0.113017,-0.059008,1.759883,15.0
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779,15.0
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711,15.0
8,7.000000,7.000000,7.000000,7.000000,7.000000,14.0


In [68]:
#these can also be used to enlarge the object in-place provided the indexer is missing
#as we can notice from the output this method seems to have some issues and it would be better to use .loc[index, col] = value instead
df.at[0, 'echo'] = 6
df

,0,2,4,6,weights,alpha,echo
0,-1.419713,1.237041,-0.186854,0.108618,0.456425,15.0,6.0
12,0.991108,-0.558594,0.113017,-0.059008,1.759883,15.0,NaN
24,-0.078035,-0.418217,2.084870,-0.695691,0.707779,15.0,NaN
36,-0.509459,-0.280707,-0.076181,0.847664,1.248711,15.0,NaN
8,7.000000,7.000000,7.000000,7.000000,7.000000,14.0,NaN


# Boolean indexing

Used to filter data with the following operations: | for or, & for and, ~ for not. These operations must be used with parentheses or risk misinterpretation by default python.

In [69]:
#finding s greater than 3
s[s> 3]

e     4
f     5
g     6
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
p    15
dtype: int64

In [70]:
#finding s less than 2 or greater than 6, remember to use parantheses
s[(s<2) | (s> 6)]

a     0
b     1
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
p    15
dtype: int64

In [71]:
# selecting s not less than 6, use ()
s[~(s<6)]

g     6
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
p    15
dtype: int64

In the case of dataframes you can select rows using a boolean vector of the same length as the index (e.g. using one of the dataframes columns)

In [72]:
#selecting all rows where column 0 is greater than 0
df[df[0]> 0]

,0,2,4,6,weights,alpha,echo
12,0.991108,-0.558594,0.113017,-0.059008,1.759883,15.0,NaN
8,7.000000,7.000000,7.000000,7.000000,7.000000,14.0,NaN


more complex operations can be accomplished using list comprehension and map methods.

In [73]:
df = pd.DataFrame({'alpha': ['blue', 'green', 'butterfly', 'bombshell'],
                  'beta' :[1, 2, 3, 4],
                  'gamma':['brocolli', 'cauliflower', 'asparagus', 'coconut']})
df

,alpha,beta,gamma
0,blue,1,brocolli
1,green,2,cauliflower
2,butterfly,3,asparagus
3,bombshell,4,coconut


In [74]:
#using a map method, selecting df where column alpha observations start with b
selection_criteria = df['alpha'].map(lambda x: x.startswith('b'))
df[selection_criteria]

,alpha,beta,gamma
0,blue,1,brocolli
2,butterfly,3,asparagus
3,bombshell,4,coconut


it is possible to use boolean vectors combined with other indexing expressions when using selection by label (.loc), selection by position(.iloc) and advanced indexing.

# Indexing with isin
the isin() method of a series accepts a list and returns a boolean vector that is is True wherever the series elements exist in the passed list. This provides a method for selecting rows where multiple comlumns contain values of interest.

In [75]:
# selecting rows containing 4, 6, or 8
s.isin([4, 6, 8])

a    False
b    False
c    False
d    False
e     True
f    False
g     True
h    False
i     True
j    False
k    False
l    False
m    False
n    False
o    False
p    False
dtype: bool

In [76]:
#pulling out rows where the previous condition is true
s[s.isin([4, 6, 8])]

e    4
g    6
i    8
dtype: int64

In [77]:
#we can use the same method on index objects in cases where we don't know if the labels we need are present
s[s.index.isin(['f', 'j', 'm'])]

f     5
j     9
m    12
dtype: int64

In [78]:
# we can also utilize this method with multiindex to check membership
s_mi = pd.Series(np.arange(12), index =  pd.MultiIndex.from_product([[0, 1, 2], ['a', 'b', 'c', 'd']]))
#checking membership
s_mi.iloc[s_mi.index.isin([(0, 'a'), (2, 'd')])]
#using the level argument
s_mi.iloc[s_mi.index.isin(['b', 'c'], level = 1)]

0  b     1
   c     2
1  b     5
   c     6
2  b     9
   c    10
dtype: int32

The isin() method is also present in dataframes where is can be passed either an array or a dict of values. A dataframe of booleans, true wherever the element is in the sequence of values, in the same shape as the original dataframe will be returned when isin is passed an array

In [79]:
# the values we are looking for are 'butterfly', 2, and 'cauliflower'
values = ['butterfly', 2, 'cauliflower']
df.isin(values)

,alpha,beta,gamma
0,False,False,False
1,False,True,True
2,True,False,False
3,False,False,False


To match certain values with certain columns you will want to make values a dict where the key is the column and the value is a list of items to check for.

In [80]:
df

,alpha,beta,gamma
0,blue,1,brocolli
1,green,2,cauliflower
2,butterfly,3,asparagus
3,bombshell,4,coconut


In [81]:
values = {'alpha': ['blue', 'green'],'beta':[2, 3], 'gamma': ['cauliflower','asparagus', 'coconut']}
df.isin(values)

,alpha,beta,gamma
0,True,False,False
1,True,True,True
2,False,True,True
3,False,False,True


In [82]:
#if we combine isin() with any() and all() methods, we can quickly subset data matched to our criteria
#using our previous values, we can see this only returns rows where all criteria are met
row_mask = df.isin(values).all(1)
df[row_mask]

,alpha,beta,gamma
1,green,2,cauliflower


# the where() method and Masking

When making a selection using a boolean vector most often you will return a subset of the data. To return selection output that is the same shape as the original we will use the where method on a series or dataframe

In [83]:
#for example, if we only want a slice
s[s>5]

g     6
h     7
i     8
j     9
k    10
l    11
m    12
n    13
o    14
p    15
dtype: int64

In [84]:
#if we want a series the same shape as the input, such as in situations where the inputs all have to be the same length
#The NumPy NaN value will be substituted where the selection condition is not met.
s.where(s>5)

a     NaN
b     NaN
c     NaN
d     NaN
e     NaN
f     NaN
g     6.0
h     7.0
i     8.0
j     9.0
k    10.0
l    11.0
m    12.0
n    13.0
o    14.0
p    15.0
dtype: float64

The where method is now built in under the hood when working with dataframes. Using a boolean vector will now return an object of the same shape with NaN's where the selection condition is not met.

In [85]:
#construct a new dataframe
df = pd.DataFrame(np.random.randn(5, 5), index = ['a', 'b', 'c', 'd', 'e'], columns = list('ABCDE'))
df

,A,B,C,D,E
a,0.529681,1.775569,0.956928,0.914167,-0.403476
b,-0.956971,0.363941,0.208228,0.708465,0.483180
c,0.116483,-0.082756,-0.234885,-0.822019,0.226901
d,1.959027,-0.235208,-0.559525,-0.218397,0.235541
e,1.036732,-1.072175,0.928715,-0.583553,-0.186007


In [86]:
#making our first selection using a boolean vector
df[df>=0]

,A,B,C,D,E
a,0.529681,1.775569,0.956928,0.914167,NaN
b,NaN,0.363941,0.208228,0.708465,0.483180
c,0.116483,NaN,NaN,NaN,0.226901
d,1.959027,NaN,NaN,NaN,0.235541
e,1.036732,NaN,0.928715,NaN,NaN


Another feature is that the where method takes an optional argument, 'other,' to specify how values should be replaced, when NaN is undesireable, in the returned copy when the condition is not met.

e.g. df.where(vector, other)

In [87]:
#for my dataframe, we're going to square all values and then perform a selection
df = df**2
df.where(df<1, 0)

,A,B,C,D,E
a,0.280562,0.000000,0.915712,0.835701,0.162793
b,0.915793,0.132453,0.043359,0.501923,0.233463
c,0.013568,0.006848,0.055171,0.675716,0.051484
d,0.000000,0.055323,0.313068,0.047697,0.055479
e,0.000000,0.000000,0.862512,0.340534,0.034599


In [88]:
#we can also set values using boolean vectors as well
df2 = df.copy()
df2[df2<1] = np.nan
df2

,A,B,C,D,E
a,NaN,3.152645,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN
d,3.837788,NaN,NaN,NaN,NaN
e,1.074814,1.149560,NaN,NaN,NaN


The where method, by default, returns a modified copy of the data. If we want to modify the original data we can use the optional inplace parameter.

In [89]:
#modifying the original dataframe
df_orig = df.copy()
#making our selection with the inplace parameter
df_orig.where(df <1, 1, inplace = True)
df_orig

,A,B,C,D,E
a,0.280562,1.000000,0.915712,0.835701,0.162793
b,0.915793,0.132453,0.043359,0.501923,0.233463
c,0.013568,0.006848,0.055171,0.675716,0.051484
d,1.000000,0.055323,0.313068,0.047697,0.055479
e,1.000000,1.000000,0.862512,0.340534,0.034599


# Alignment with the where method
This method takes the input and aligns the input in a way that allows you to both select a subset and set values for a subset at the same time. This is similar to partial setting using the .loc accessor which acts on the axis labels as opposed to the where method which acts on the contents.

In [100]:
#create our working copy
df2 = df.copy()
#perform our boolean selection and set values based on that condition
df2[df2>1] = 0.5
df2

,A,B,C,D,E
a,0.280562,0.500000,0.915712,0.835701,0.162793
b,0.915793,0.132453,0.043359,0.501923,0.233463
c,0.013568,0.006848,0.055171,0.675716,0.051484
d,0.500000,0.055323,0.313068,0.047697,0.055479
e,0.500000,0.500000,0.862512,0.340534,0.034599


we can also utilize the axis and level parameters for input alignment when performing the where. For example:

In [99]:
#perform a selection and specifying the axis. Since we are not using a multindex specifying level is unneccessary
df2.where(df2 == 0.5, df2**2, axis = 'index')

,A,B,C,D,E
a,0.078715,0.500000,0.838528,0.698396,0.026501
b,0.838677,0.017544,0.001880,0.251927,0.054505
c,0.000184,0.000047,0.003044,0.456592,0.002651
d,0.500000,0.003061,0.098012,0.002275,0.003078
e,0.500000,0.500000,0.743927,0.115964,0.001197


We can perform the same operation using the appply method and a lambda function but the where method is faster and presumably more computationally efficient.

As a new feature the where method can also accept a callable as condition and other arguments. It must contain only one argument and it must return a valid argument.

In [104]:
#using a callable as both condition and other arguments
#we are going to take the cube root of all values where the dataframe equals 0.5
df2.where(lambda x: x!=0.5, lambda x: x**-3)

,A,B,C,D,E
a,0.280562,8.000000,0.915712,0.835701,0.162793
b,0.915793,0.132453,0.043359,0.501923,0.233463
c,0.013568,0.006848,0.055171,0.675716,0.051484
d,8.000000,0.055323,0.313068,0.047697,0.055479
e,8.000000,8.000000,0.862512,0.340534,0.034599


# Mask

this is the inverse of the boolean operation where.
what does that mean?
I believe this means that for every instance the passed condition evaulates to true, mask selects the observations where it isn't

In [106]:
#in the case of a series
#we will be masking all values not equal to 6
s.mask(s!=6)

a    NaN
b    NaN
c    NaN
d    NaN
e    NaN
f    NaN
g    6.0
h    NaN
i    NaN
j    NaN
k    NaN
l    NaN
m    NaN
n    NaN
o    NaN
p    NaN
dtype: float64

In [108]:
#in the case of a DataFrame
#we will be masking all values less than or equal to 0.5
df.mask(df<=0.5)

,A,B,C,D,E
a,NaN,3.152645,0.915712,0.835701,NaN
b,0.915793,NaN,NaN,0.501923,NaN
c,NaN,NaN,NaN,0.675716,NaN
d,3.837788,NaN,NaN,NaN,NaN
e,1.074814,1.149560,0.862512,NaN,NaN


# The Query method

This is specific to DataFrames. This method allows us to make a selection using expressions.

For example, if we want to select all rows in column B that are greater than or equal to column C we can do the following

In [110]:
#using our most recent df
df.query('(B>=C)')

,A,B,C,D,E
a,0.280562,3.152645,0.915712,0.835701,0.162793
b,0.915793,0.132453,0.043359,0.501923,0.233463
e,1.074814,1.149560,0.862512,0.340534,0.034599


# come back to this